In [9]:
import pandas as pd
import numpy as np
import os

from sklearn import *

DATA_FOLDER = '../readonly/final_project_data/'

train           = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
test            = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
sample          = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv.gz'))

sample.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [2]:
train.shape

(2935849, 6)

In [3]:
test.shape

(214200, 3)

In [4]:
test_shops = test.shop_id.unique()
train = train[train.shop_id.isin(test_shops)]
test_items = test.item_id.unique()
train = train[train.item_id.isin(test_items)]

train.shape

(1224439, 6)

In [5]:
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year

train = train.drop(['date','item_price'], axis=1)
train = train.groupby([c for c in train.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
train = train.rename(columns={'item_cnt_day':'item_cnt_month'})

train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month
0,0,2,33,1,2013,1.0
1,0,2,482,1,2013,1.0
2,0,2,491,1,2013,1.0
3,0,2,839,1,2013,1.0
4,0,2,1007,1,2013,3.0


In [6]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34
test['item_cnt_month'] = 0.0

test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month
0,0,5,5037,11,2015,34,0.0
1,1,5,5320,11,2015,34,0.0
2,2,5,5233,11,2015,34,0.0
3,3,5,5232,11,2015,34,0.0
4,4,5,5268,11,2015,34,0.0


In [7]:
col = [c for c in train.columns if c not in ['item_cnt_month']]

x1 = train[train['date_block_num']<33]
y1 = np.log1p(x1['item_cnt_month'].clip(0.,20.))
x1 = x1[col]
x2 = train[train['date_block_num']==33]
y2 = np.log1p(x2['item_cnt_month'].clip(0.,20.))
x2 = x2[col]

reg = ensemble.ExtraTreesRegressor(n_estimators=25, n_jobs=-1, max_depth=15, random_state=18)
reg.fit(x1,y1)

print('RMSE:', np.sqrt(metrics.mean_squared_error(y2.clip(0.,20.),reg.predict(x2).clip(0.,20.))))

reg.fit(train[col],train['item_cnt_month'].clip(0.,20.))

test['item_cnt_month'] = reg.predict(test[col]).clip(0.,20.)

test.head()

RMSE: 0.403247461329


,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month
0,0,5,5037,11,2015,34,1.585524
1,1,5,5320,11,2015,34,1.403576
2,2,5,5233,11,2015,34,1.454750
3,3,5,5232,11,2015,34,1.400632
4,4,5,5268,11,2015,34,1.393966


In [8]:
test[['ID','item_cnt_month']].to_csv('submission.csv', index=False)